## Librerías necesarias

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
import os, shutil, pathlib
from tensorflow.keras.utils import image_dataset_from_directory
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

## Variables globales del sistema

In [ ]:
original_dir = pathlib.Path("DatasetOriginal/Image Data Base")
new_base_dir = pathlib.Path("TFGDataset")
content = os.listdir(original_dir)
BATCH_SIZE = 64
size_x = 256
size_y = 256

## Función para contar imágenes por categorías

In [ ]:
def contando_categorias():
    counter_more200 = 0
    counter_less200 = 0
    categories = {}
    for category in content:
        counter = 0
        for path in pathlib.Path(original_dir / category).iterdir():   
            counter += 1
        if counter < 200:
            counter_less200 += 1
            categories.update({str(category):counter})
            
        else:
            counter_more200 += 1
    print("Hay "+ str(counter_more200) + " categorias con más de 200 ejemplos")
    print("Hay "+ str(counter_less200) + " categorias con menos de 200 ejemplos")
    return categories
        
categories = contando_categorias()

In [ ]:
print(categories)

## Función para crear carpetas de subconjuntos de datos a partir de un dataset

In [ ]:
def make_subsets(proportion, max_data):
    for category in content:
        fnames = []
        count = 1
        for path in (original_dir / category).iterdir():
            fnames.append(os.path.basename(path))
            count += 1
        leftover = count % 10
        data = count - leftover
        data = min(data, max_data)
        primero = int(data * proportion[0])
        segundo = int(data * proportion[1]) + primero
        tercero = int(data * proportion[2]) + segundo
        train = fnames[0:primero]
        validation = fnames[primero:segundo]
        test = fnames[segundo:tercero]
        if data < 200 and leftover > 0:
            train.extend(fnames[tercero:count - 1])
        classification = {"train": train, "validation": validation, "test": test}
        for subset in ["train", "validation", "test"]:
            dir = new_base_dir / subset / category
            os.makedirs(dir)
            count = 1
            for fname in classification[subset]:
                file = str(count) + '.jpg'
                try:
                    shutil.copyfile(src=original_dir / category / fname, dst=dir / file)
                    count += 1
                except:
                    print("error")
                    
                    
proportion = [0.8, 0.1, 0.1] #Para dividir los datos en conjuntos de estas proporciones
make_subsets(proportion, 200)

## Creación de los datasets específicos a partir de las carpetas 

In [ ]:
train_dataset = image_dataset_from_directory(
    new_base_dir / "train",
    image_size=(size_x, size_y),
    batch_size=BATCH_SIZE,
    label_mode='categorical')
validation_dataset = image_dataset_from_directory(
    new_base_dir / "validation",
    image_size=(size_x, size_y),
    batch_size=BATCH_SIZE,
    label_mode='categorical')
test_dataset = image_dataset_from_directory(
    new_base_dir / "test",
    image_size=(size_x, size_y),
    batch_size=BATCH_SIZE,
    label_mode='categorical')

## Comprobación de que está todo en orden y muestreo

In [ ]:
 for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

In [ ]:
plt.figure(figsize=(13, 13))
class_names = train_dataset.class_names
for images, labels in train_dataset.take(2):
    for i in range(3):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        arr = labels[i].numpy()
        tuple = np.where(arr == 1)
        plt.title(class_names[tuple[0][0]])
        plt.axis("off")

## Modelo creado desde cero

In [ ]:
inputs = keras.Input(shape=(size_x, size_y, 3)) 
x = layers.Rescaling(1./255)(inputs) 
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(58, activation="softmax")(x)
model0 = keras.Model(inputs=inputs, outputs=outputs)

model0.summary()

## Entrenamiento y guardado de resultados del modelo cero

In [ ]:
callbacks = [
 keras.callbacks.ModelCheckpoint(
 filepath="model0/best_version",
 save_best_only=True,
 monitor="val_loss")
]

In [ ]:
model0.compile(loss="categorical_crossentropy",
 optimizer="rmsprop",
 metrics=["accuracy"])

In [ ]:
history = model0.fit(
 train_dataset,
 epochs=30,
 validation_data=validation_dataset,
 callbacks=callbacks)

In [ ]:
model0.save('model0/model')

In [ ]:
np.save('model0/history.npy',history.history)

## Análisis de resultados

In [ ]:
my_model = load_model('model0/model')

In [ ]:
my_history=np.load('model0/history.npy',allow_pickle='TRUE').item()

In [ ]:
loss = my_history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
plt.clf()
acc = my_history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

In [ ]:
model = keras.models.load_model('model0/model')
results= model.evaluate(test_dataset)
results

In [ ]:
model = keras.models.load_model('model0/best_version')
results= model.evaluate(test_dataset)
results